In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

---
1. Total Scope 1 and 2 Emissions (tons of CO2e): This is the dependent variable that represents the combined direct and indirect greenhouse gas emissions.
2. Charlie (in mil USD): Represents costs incurred on the emission.
3. Romeo (mil USD): Represent production value of emission.
4. Echo (Number): The total number of community members affected.
5. Alpha (mil USD): Total market capital of the company.
---

#### Methodology for Model Development

1. Describe the methodology applied for developing Scope 1 and 2 emission estimation models.
2. Justify the choice of the model/methodology using statistical logic and reasoning.
---
#### Model Results Summary
1. Provide an overview and summary of the results obtained from the developed models.
Model Validation
2. Briefly explain the model validation process.
3. Justify the choice of metrics or validation process used, highlighting their relevance and effectiveness.

---

#### Sector Analysis
1. Identify and analyze which sectors have higher Scope 1 and 2 emissions and which have lower
emissions.
2. Explain the reasons behind the differences in emission levels across sectors.
---
#### Data Requirements for Model Improvement
1. Identify additional data that could improve the accuracy and reliability of the emission
estimation models.
2. Explain how the additional data would enhance the models and lead to better results.o better results.

In [36]:
data_path = r"E:\Workspace\Data Science\Data Science Project\scope_1_scope_2_emmisions\ISSCaseStudy_Sample data for modelling.csv"

df = pd.read_csv(data_path)
df["Total S1+S2 (tCo2e)"] = df["Total S1+S2 (tCo2e)"].astype(int)
df.head(20)

,Company Identifier,Total S1+S2 (tCo2e),Subsector Name,Charlie (milUSD),Alpha (milUSD),Romeo (milUSD),Echo
0,1,6399,Software,NaN,NaN,NaN,NaN
1,2,59990,Software,1270.020324,1061.630736,8869.070062,21357.0
2,3,296160,Specialty Chemicals,6292.864579,2018.705686,10689.439260,34500.0
3,4,960000,Specialty Chemicals,2107.656246,2956.230233,3302.499773,5900.0
4,5,253641,Health Care Services,8208.619341,1746.387752,11090.105310,61000.0
5,6,5257,Diversified Banks,NaN,381.105765,3684.022390,9831.0
6,7,9690000,Software,136168.730900,60470.400590,227887.602700,647500.0
7,8,7821702,Oil & Gas Exploration & Production,5170.568097,15737.745400,6008.192301,1708.0
8,9,12487,Capital Markets,1712.726219,-458.638393,5126.240555,7459.0
9,10,1010030,Specialty Chemicals,2724.917214,1874.182466,3694.083713,6000.0


#### Data Preprocessing

- Since there are 872 rows in a dataframe and out of which charlie has the the most number of NaN's followed by 75 in Alpha, 64 in Alpha and 119 in Echo, so dropping the value would significantly result in data loss.
- There are multiple ways to handle NaN's but I am going with filling those NaN's with their respective sectore wise median value

In [37]:
## Nan values per column
df.isna().sum()

Company Identifier       0
Total S1+S2 (tCo2e)      0
Subsector Name           0
Charlie (milUSD)       257
Alpha (milUSD)          75
Romeo (milUSD)          64
Echo                   119
dtype: int64

In [38]:
sector_medians = df.groupby("Subsector Name")[["Charlie (milUSD)",
                             "Alpha (milUSD)","Romeo (milUSD)",
                             "Echo"]].median().T.to_dict()

In [39]:
def fill_missing_values(row):
    sector = row['Subsector Name']
    for col in ["Charlie (milUSD)","Alpha (milUSD)","Romeo (milUSD)","Echo"]:
        if pd.isna(row[col]) and sector in sector_medians:
            row[col] = sector_medians[sector][col]
    return row

df = df.apply(fill_missing_values, axis = 1)

In [40]:
# Create new features
df['Energy_Intensity'] = df['Total S1+S2 (tCo2e)'] / df['Charlie (milUSD)']
df['Emissions_per_Production'] = df['Total S1+S2 (tCo2e)'] / df['Romeo (milUSD)']
df['Raw_Material_Intensity'] = df['Total S1+S2 (tCo2e)'] / df['Alpha (milUSD)']

# Normalize features by sector medians
for sector in sector_medians.keys():
    mask = df['Subsector Name'] == sector
    df.loc[mask, 'Normalized_Charlie'] = df.loc[mask, 'Charlie (milUSD)'] / sector_medians[sector]['Charlie (milUSD)']
    df.loc[mask, 'Normalized_Alpha'] = df.loc[mask, 'Alpha (milUSD)'] / sector_medians[sector]['Alpha (milUSD)']
    df.loc[mask, 'Normalized_Romeo'] = df.loc[mask, 'Romeo (milUSD)'] / sector_medians[sector]['Romeo (milUSD)']
    df.loc[mask, 'Normalized_Echo'] = df.loc[mask, 'Echo'] / sector_medians[sector]['Echo']

# Relative Emission Intensity
sector_emissions_median = df.groupby('Subsector Name')['Total S1+S2 (tCo2e)'].median().to_dict()
df['Relative_Emission_Intensity'] = df.apply(lambda x: x['Total S1+S2 (tCo2e)'] / sector_emissions_median[x['Subsector Name']], axis=1)


In [47]:
## Sector wise Analysis
# High emission sectors
df.groupby("Subsector Name")["Total S1+S2 (tCo2e)"].sum().sort_values(ascending = False)

Subsector Name
Cement Manufacturers                  960409885
Oil & Gas Refining & Marketing        247883691
Aluminum                              227638917
Oil & Gas Exploration & Production    149710757
Industrial Conglomerates              110548985
Paper & Related Products               98692778
Automobiles                            61877418
Oil & Gas Storage & Transportation     59853573
Specialty Chemicals                    45146452
Coal                                   42543650
Waste & Disposal Services              36376858
Auto Parts                             31185275
Hotels                                 19564725
Oil & Gas Equipment & Services         16139218
Software                               12615056
Diversified Banks                      11577758
Computer Storage & Peripherals          5716451
Gambling                                5326704
Health Care  Services                   3443915
Capital Markets                         1908997
Regional Banks           

---
### Top 5 Subsector Emissions:
- Cement Manufacturers
- Oil & Gas Refining & Marketing
- Aluminum
- Oil & Gas Exploration & Production
- Industrial Conglomerates
---
### Last 5 Subsector Emissions:
- Regional Banks
- Capital Markets
- Health Care Services
- Gambling
- Computer Storage & Peripherals
---
- __Operational Differences: Industries like Specialty Chemicals, Cement Manufacturing, and Oil & Gas involve processes that inherently produce more emissions, such as chemical reactions, high heat requirements, and direct fossil fuel use.__
- __Energy Efficiency: Sectors with lower emissions, such as Software, often benefit from more energy-efficient operations and less dependence on heavy industrial equipment.__